# Gradient Boosting Trees-model: XGBoost

I denne notebook vil vi afprøve en model med en [*XGBoost*](https://xgboost.readthedocs.io/en/stable/)-predictor, der tilhører familien af &#128073; [*Gradient Boosting Trees*-predictors](slides/08_xgboost.ipynb).

## Import af python-afhængigheder

In [ ]:
from sklearn.model_selection import ShuffleSplit, GridSearchCV
from xgboost import XGBClassifier, plot_importance
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, roc_curve
from feature_engine.selection import (
    DropConstantFeatures,
)
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from workshop.utils import DropFeatures

## Indlæs data

In [ ]:
# indlæs "rådata" for 2015-18
df = pd.read_parquet("data/train.parquet")

# dan binomial "target"-variabel, der angiver, om der har været skade(r) eller ej
y = df["skadesantal"].values > 0

# instantiér generator til split af data, random state sættes for at kunne reproducere split
split_generator = ShuffleSplit(n_splits=1, train_size=0.8, random_state=42)

# generér indices for de to splits
train_index, val_index = next(split_generator.split(df))

# opdel data i trænings- og valideringssæt
X_train, y_train = df.iloc[train_index], y[train_index]
X_val, y_val = df.iloc[val_index], y[val_index]

## Data-transformationer

*XGBoost*-predictoren kan håndtere de fleste inputs. Den kan håndtere kategoriske variable og missing-værdier *under the hood*.

Derfor bliver vores data-transformations-pipeline minimal.

In [ ]:
blacklist = [
    "aar",
    "idnummer",
    "skadesudgift",
    "skadesantal",
]

pipe = Pipeline(
    [
        ("drop_features_blacklist", DropFeatures(blacklist)),
        ("drop_constant_features", DropConstantFeatures(missing_values="ignore")),
    ]
)

# vis pipeline
pipe

## Model-træning
Vi anvender konkret predictoren [`XGBClassifier`](https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.XGBClassifier) fra `xgboost`. Vi instantierer predictoren med dens default-parameter-værdier. 

Vi udvider vores data-transformations-pipeline med predictoren.

In [ ]:
# instantiér predictor
predictor = XGBClassifier(
    objective="binary:logistic",
    enable_categorical=True,
)

# udvid pipeline med vores predictor
pipe.steps.append(["predictor", predictor])

# visualize pipeline
pipe

Pipeline inkl. predictor fittes under ét på vores træningsdata.

In [ ]:
pipe.fit(X_train, y_train);

## Model-evaluering

Vi plotter ROC-kurverne for modellen på trænings- og valideringssættet.

In [ ]:
# beregn true positive rate og false positive rate for trænings- og valideringssæt
ns_fpr, ns_tpr, _ = roc_curve(y_train, [0 for _ in range(len(y_train))])
train_fpr, train_tpr, _ = roc_curve(y_train, pipe.predict_proba(X_train)[:, 1])
test_fpr, test_tpr, _ = roc_curve(y_val, pipe.predict_proba(X_val)[:, 1])

# plot ROC-kurver
plt.plot(ns_fpr, ns_tpr, linestyle="--", label="Naïve")
plt.plot(train_fpr, train_tpr, label="Train")
plt.plot(test_fpr, test_tpr, label="Validation")

# formatér og display plot
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.show()

Vi bemærker, at ROC-kurven for træningssættet ser markant bedre ud end den for valideringssættet.

Vi beregner for begge sæt [*ROC AUC*](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score).




In [ ]:
# beregn ROC AUC på både trænings- og valideringssæt
auc_train = roc_auc_score(y_train, pipe.predict_proba(X_train)[:, 1])
auc_val = roc_auc_score(y_val, pipe.predict_proba(X_val)[:, 1])
print("ROC AUC train: ", auc_train, "\nROC AUC val: ", auc_val)

Det bemærkes, at :

1. Modellen performer signifikant bedre end en naiv model på både trænings- og valideringssæt
2. Modellen performer voldsomt meget bedre på træningssættet end på valideringssættet. Det er et klart tegn på *overfitting*.

### Model-diagnostik

`XGBoost` har nyttige indbyggede værktøjer til at beregne og plotte feature importance.

In [ ]:
plot_importance(pipe.named_steps["predictor"], max_num_features=5)

**Bemærk**, hvor parametrene i en `LogisticRegression` har en pæn fortolkning, går forklarligheden fløjten i en black-box-model som `xgboost`.

Vi inspicerer fordelingen af de prædikterede sandsynligheder.

In [ ]:
sns.histplot(pipe.predict_proba(X_train)[:, 1])
plt.xlim(0, 0.1)

Vi ser igen en "pæn" fordeling af prædiktionerne.

## Model tuning
Som I kan se af dokumentationen for [`XGBClassifier`](https://xgboost.readthedocs.io/en/stable/parameter.html#parameters-for-tree-booster), er der voldsomt mange håndtag, man kan skrue på i bestræbelserne på at optimere modellens prædiktive performance.

Vi vælger at tune parametrene *learning_rate*, *max_depth* og *n_estimators* simultant.

Vi anvender et simpelt `GridSearch` til formålet.

In [ ]:
# Definér parameter-grid, der skal gennemsøges
param_grid = {
    "predictor__learning_rate": [0.1, 0.3],
    "predictor__max_depth": [3, 6],
    "predictor__n_estimators": [100, 250],
}

# Forbered GridSearch
# Bemærk, at vi her bruger vores split_generator til at generere trænings- og valideringssæt
grid = GridSearchCV(
    pipe,
    param_grid,
    cv=split_generator,
    scoring="roc_auc",
    return_train_score=True,
    n_jobs=-1,
    verbose=4,
)

# Udfør grid search - bemærk, vi giver hele datasættet (2015-18) som input og overlader det til split-generatoren at splitte det i trænings- og valideringssæt
grid.fit(df, y)

print("Disse parametre giver den bedste performance: ", grid.best_params_)
print("Med en ROC AUC på: ", grid.best_score_)

# Udskriv resultater
results = pd.DataFrame.from_records(grid.cv_results_["params"])
results["mean_valid_score"] = grid.cv_results_["mean_test_score"]
results["mean_train_score"] = grid.cv_results_["mean_train_score"]
print("Resultater for grid search:")
results.sort_values("mean_valid_score", ascending=False)

På nettet kan man finde mange bud på, hvordan `xgboost` skal tunes, f.eks. [dette](https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/).

### Kan I gøre det bedre?

&#128073; [Model Lab](slides/09_kaggle.ipynb)